In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from tqdm.notebook import tqdm
from pathlib import Path
import re
import time

## Data Scrapping

#### 1. 

In [3]:
# set up the path 
data_path = Path.cwd().parent / 'data'

data_path.mkdir(exist_ok=True)
print(data_path)


/Users/zoeyy/Desktop/xlab Intern/ds-ciss-colonial-law/data


In [233]:


# Function to scrape data from a given URL
def scrape_data(year, id_range):
    data = []
    for i in id_range:
        url = f"https://chanrobles.com/cralaw/{year}januarydecisions.php?id={i}"
        print(f"Scraping URL: {url}")
        
        # Define headers to mimic a browser request
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }

        try:
            # Send a GET request to the webpage with headers and timeout
            response = requests.get(url, headers=headers, timeout=10)
            
            # If status code 404, break the loop
            if response.status_code == 404:
                print(f"No more data available for {year} with id={i}")
                break

            # If any other status code, continue to the next ID
            if response.status_code != 200:
                print(f"Failed to retrieve the webpage for {year} with id={i}. Status code: {response.status_code}")
                continue

            # Parse the webpage content
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find the relevant section containing the case details
            main_content = soup.find('div', class_='topcontent')
            if not main_content:
                continue

            # Initialize variables to store case details
            case_details = {'year': year, 'url': url, 'id': i}

            # Find the date, plaintiffs, and defendants from the justified content
            justify_div = main_content.find('div', align='justify')
            if justify_div:
                text = justify_div.get_text(separator=' ', strip=True)

                parts = re.split(r' - |\]', text)
                
                if len(parts) >= 2:
                    date_part = parts[0].strip()
                    case_info = parts[1].strip()
                else:
                    date_part = parts[0].strip()
                    case_info = ''

                # Extract date and case number from date_part
                date_match = re.search(r'(\w+ \d{1,2}, \d{4})', date_part)
                if date_match:
                    case_details['date'] = date_match.group(1)
                
                case_id_match = re.search(r'G\.? ?R\.? No\.? ?(\S+)', date_part)
                if case_id_match:
                    case_details['case_NO'] = case_id_match.group(1)

                    case_parts = re.split(r' v\. | vs\. ', case_info)
                    if len(case_parts) == 2:
                        case_details['plaintiffs'] = case_parts[0].strip()
                        
                        # Remove extra text after the defendant's name
                        defendant_info = case_parts[1].strip()
                        defendant_name = re.match(r'^[^\d]+', defendant_info)
                        if defendant_name:
                            case_details['defendants'] = defendant_name.group(0).strip()
                        else:
                            case_details['defendants'] = defendant_info
            
            # Extract the lawyer for plaintiff
            center_divs = main_content.find_all('div', align='center')[1]
            lawyers = center_divs.find_all(string=re.compile(r', for'))
            if lawyers and len(lawyers[0].strip(', for'))<50:
                case_details['lawyer_for_plaintiff'] = lawyers[0].strip(', for')
                if len(lawyers) > 1:
                    case_details['lawyer_for_defendant'] = lawyers[1].strip(', for')
            else:
                lawyers_text = center_divs.find_all(string=re.compile(r' for '))
                if lawyers_text:
                    case_details['lawyer_for_plaintiff'] = lawyers_text[0].strip(' for ')
                    if len(lawyers_text) > 1:
                        case_details['lawyer_for_defendant'] = lawyers_text[1].strip(' for ')

            # Extract syllabus
            syllabus_tag = main_content.find(string="SYLLABUS")
            if syllabus_tag:
                case_details['syllabus'] = syllabus_tag.find_next(string=True).strip()
            else:
                case_details['syllabus'] = "N/A"
            
            # Extract the content under align='justify' after decision
            decision_tag = main_content.find(string="D E C I S I O N")
            if decision_tag:
                writer_content = decision_tag.find_next(lambda tag: tag.name in ['p', 'div'] and tag.get('align') == 'right')
                if writer_content:
                    writer = writer_content.get_text(separator=' ', strip=True)
                    print(writer)
                    # match the text up to and including 'J'. 
                    match = re.match(r'.*?J\.?', writer)
                    if match and len(match.group(0)) > 5:
                        case_details['writer'] = match.group(0)
                    else:
                        case_details['writer'] = writer
                    
                justify_content = decision_tag.find_next('div', align='justify')
                if justify_content:
                    case_details['decision'] = justify_content.get_text(separator=' ', strip=True)
                else:
                    p_tags = decision_tag.find_all_next('p', style=re.compile(r'margin: 6pt 0in; text-align: justify; line-height: normal;'))
                    if p_tags:
                        decision_text = ' '.join(p.get_text(separator=' ', strip=True) for p in p_tags)
                        case_details['decision'] = decision_text

                # Find the concur tag
                concur_tag = soup.find(string=re.compile(r'concur\.'))
                concur_text = ""

                # Extract the text before "concur."
                if concur_tag:
                    full_text = concur_tag.parent.get_text(separator=' ', strip=True)
                    concur_text = re.search(r'(.*concur\.)', full_text).group(1).strip()
                    if len(concur_text) < 200:
                        case_details['concur'] = concur_text
                    else:
                        # the concur text is too long, extract the last 100 characters
                        case_details['concur'] = concur_text[-100:]

                else:
                    case_details['concur'] = "N/A"

            # Extract separate opinions
            separate_opinion = main_content.find(string="Separate Opinions")
            if separate_opinion:
                case_details['separate_opinion'] = True
            else:
                case_details['separate_opinion'] = False

            # Extract endnotes  
            endnotes = main_content.find(string="Endnotes:")
            if endnotes:
                case_details['endnotes'] = True
            else:
                case_details['endnotes'] = False
            
            data.append(case_details)
        
        except requests.exceptions.Timeout:
            print(f"Request timed out for URL: {url}")
            continue
        except requests.exceptions.RequestException as e:
            print(f"Failed to retrieve the webpage for URL: {url}. Error: {e}")
            continue

    return data


In [234]:
# Define the range of years and ID range
years = range(1901, 1947)  
id_range = range(1,500)

# Initialize a list to store all scraped data
all_data = []

# Scrape data for each year
for year in tqdm(years, total=len(years)):
    data = scrape_data(year, id_range)
    all_data.extend(data)

# Create a DataFrame from the scraped data
df = pd.DataFrame(all_data)
df.head(10)

#df_clean = df.drop_duplicates(inplace = True)
#print(f"'{df_clean.shape[0]}' rows, without duplicates")


  0%|          | 0/46 [00:00<?, ?it/s]

Scraping URL: https://chanrobles.com/cralaw/1901januarydecisions.php?id=1
Scraping URL: https://chanrobles.com/cralaw/1901januarydecisions.php?id=2
Scraping URL: https://chanrobles.com/cralaw/1901januarydecisions.php?id=3
Scraping URL: https://chanrobles.com/cralaw/1901januarydecisions.php?id=4
SMITH, J. :
Scraping URL: https://chanrobles.com/cralaw/1901januarydecisions.php?id=5
COOPER, J. :
Scraping URL: https://chanrobles.com/cralaw/1901januarydecisions.php?id=6
SMITH, J. :
Scraping URL: https://chanrobles.com/cralaw/1901januarydecisions.php?id=7
WILLARD, J. :
Scraping URL: https://chanrobles.com/cralaw/1901januarydecisions.php?id=8
WILLARD, J. :
Scraping URL: https://chanrobles.com/cralaw/1901januarydecisions.php?id=9
WILLARD, J. :
Scraping URL: https://chanrobles.com/cralaw/1901januarydecisions.php?id=10
TORRES, J. :
Scraping URL: https://chanrobles.com/cralaw/1901januarydecisions.php?id=11
LADD, J. :
Scraping URL: https://chanrobles.com/cralaw/1901januarydecisions.php?id=12
TORRES

  2%|▏         | 1/46 [01:21<1:00:48, 81.07s/it]

Scraping URL: https://chanrobles.com/cralaw/1902januarydecisions.php?id=1
TORRES, J. :
Scraping URL: https://chanrobles.com/cralaw/1902januarydecisions.php?id=2
TORRES, J. :
Scraping URL: https://chanrobles.com/cralaw/1902januarydecisions.php?id=3
ARELLANO, C.J. :
Scraping URL: https://chanrobles.com/cralaw/1902januarydecisions.php?id=4
ARELLANO, C.J. :
Scraping URL: https://chanrobles.com/cralaw/1902januarydecisions.php?id=5
MAPA, J. :
Scraping URL: https://chanrobles.com/cralaw/1902januarydecisions.php?id=6
WILLARD, J. :
Scraping URL: https://chanrobles.com/cralaw/1902januarydecisions.php?id=7
TORRES, J. :
Scraping URL: https://chanrobles.com/cralaw/1902januarydecisions.php?id=8
TORRES, J. :
Scraping URL: https://chanrobles.com/cralaw/1902januarydecisions.php?id=9
TORRES, J. :
Scraping URL: https://chanrobles.com/cralaw/1902januarydecisions.php?id=10
LADD, J. :
Scraping URL: https://chanrobles.com/cralaw/1902januarydecisions.php?id=11
TORRES, J. :
Scraping URL: https://chanrobles.com

  4%|▍         | 2/46 [10:15<4:14:50, 347.50s/it]

...

Scraping URL: https://chanrobles.com/cralaw/1944januarydecisions.php?id=499
Scraping URL: https://chanrobles.com/cralaw/1945januarydecisions.php?id=1
MORAN, C.J. :
Scraping URL: https://chanrobles.com/cralaw/1945januarydecisions.php?id=2
DE JOYA, J. :
Scraping URL: https://chanrobles.com/cralaw/1945januarydecisions.php?id=3
HILADO, J. :
Scraping URL: https://chanrobles.com/cralaw/1945januarydecisions.php?id=4
JARANILLA, J. :
Scraping URL: https://chanrobles.com/cralaw/1945januarydecisions.php?id=5
PABLO, M. :
Scraping URL: https://chanrobles.com/cralaw/1945januarydecisions.php?id=6
FERIA, J. :
Scraping URL: https://chanrobles.com/cralaw/1945januarydecisions.php?id=7
PARAS, J. :
Scraping URL: https://chanrobles.com/cralaw/1945januarydecisions.php?id=8
OZAETA, J. :
Scraping URL: https://chanrobles.com/cralaw/1945januarydecisions.php?id=9
PABLO, J. :
Scraping URL: https://chanrobles.com/cralaw/1945januarydecisions.php?id=10
DE JOYA, J. :
Scraping URL: https://chanrobles.com/cralaw/1945jan

 98%|█████████▊| 45/46 [1:21:00<01:31, 91.01s/it]

Scraping URL: https://chanrobles.com/cralaw/1946januarydecisions.php?id=1
MORAN, C.J. :
Scraping URL: https://chanrobles.com/cralaw/1946januarydecisions.php?id=2
OZAETA, J. :
Scraping URL: https://chanrobles.com/cralaw/1946januarydecisions.php?id=3
OZAETA, J. :
Scraping URL: https://chanrobles.com/cralaw/1946januarydecisions.php?id=4
PERFECTO, J. :
Scraping URL: https://chanrobles.com/cralaw/1946januarydecisions.php?id=5
OZAETA, J. :
Scraping URL: https://chanrobles.com/cralaw/1946januarydecisions.php?id=6
PARAS, J. :
Scraping URL: https://chanrobles.com/cralaw/1946januarydecisions.php?id=7
PARAS, J. :
Scraping URL: https://chanrobles.com/cralaw/1946januarydecisions.php?id=8
PABLO, M. :
Scraping URL: https://chanrobles.com/cralaw/1946januarydecisions.php?id=9
MORAN, C.J. :
Scraping URL: https://chanrobles.com/cralaw/1946januarydecisions.php?id=10
PERFECTO, J. :
Scraping URL: https://chanrobles.com/cralaw/1946januarydecisions.php?id=11
JARANILLA, J. :
Scraping URL: https://chanrobles.co

100%|██████████| 46/46 [1:22:30<00:00, 107.61s/it]


,year,url,id,syllabus,separate_opinion,endnotes,date,case_NO,lawyer_for_plaintiff,writer,decision,concur,plaintiffs,defendants,lawyer_for_defendant
0,1901,https://chanrobles.com/cralaw/1901januarydecis...,1,N/A,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1901,https://chanrobles.com/cralaw/1901januarydecis...,2,N/A,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1901,https://chanrobles.com/cralaw/1901januarydecis...,3,N/A,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1901,https://chanrobles.com/cralaw/1901januarydecis...,4,1. CONTEMPT OF COURT; TREATMENT OF WITNESS. — ...,False,False,"August 8, 1901",12,Claudio Gabriel,"SMITH, J.",It appears from the record in this matter that...,"s so ordered, with costs de oficio. Arellano, ...",NaN,NaN,NaN
4,1901,https://chanrobles.com/cralaw/1901januarydecis...,5,1. PARTNERSHIP; ACCOUNTING. — Any partner may ...,False,False,"August 24, 1901",26,Charles C . Cohn,"COOPER, J.",This is an appeal from the judgment of the Cou...,"s on appeal taxed to the Appellant . Arellano,...",WALTER JACKSON,"PAUL BLUM, ET AL.",Wm. J . Rohde
5,1901,https://chanrobles.com/cralaw/1901januarydecis...,6,1. COMMON CARRIERS; BILLS OF LADING. — Where a...,False,False,"August 26, 1901",17,Alberto Barrett,"SMITH, J.",This action was commenced in the Court of Firs...,"costs, and it is so ordered. Arellano, C.J. ,...",DON LUCIANO CORDOBA,"WARNER, BARNES & CO.",Josiah Mode Vale
6,1901,https://chanrobles.com/cralaw/1901januarydecis...,7,1. HABEAS CORPUS; JURISDICTION. — Under Genera...,False,False,"August 28, 1901",456,Eber C . Smith,"WILLARD, J.",No judge of this Archipelago has at present ju...,"d August 23, 1901, is hereby repealed. Arellan...",NaN,NaN,"Colonel Grosbeck, United States Army"
7,1901,https://chanrobles.com/cralaw/1901januarydecis...,8,1. VENUE OF ACTIONS. — An action to collect re...,False,False,"September 9, 1901",100,Jose Robles Lahesa,"WILLARD, J.",It has been plainly proven that at the time th...,against the Appellant . It so ordered. Arellan...,AGUSTIN ASENCIO,FRANCISCO GUTIERREZ,Vicente Miranda
8,1901,https://chanrobles.com/cralaw/1901januarydecis...,9,1. REAL PROPERTY; MORTGAGE LAW; REGISTRATION O...,False,False,"September 12, 1901",18,Eminio Gaudie,"WILLARD, J.",This case was initiated and carried on in reli...,erefore be reversed. It is so ordered. Arellan...,GAUDENCIO ELEIZEGUI,JOSEFA. AREVALO,Alfredo Chicote
9,1901,https://chanrobles.com/cralaw/1901januarydecis...,10,1. CRIMINAL LAW; ASSAULT; FRUSTRATED HOMICIDE....,False,False,"September 16, 1901",64,Lorenzo Albert,"TORRES, J.",On the morning of May 27 of the current year V...,subsidiary imprisonment. So ordered. Arellano...,UNITED STATES,VICENTE TAGUIBAO,Solicitor-General Araneta


In [235]:

# Save the DataFrame to a CSV file
df.to_csv(data_path/ 'scrap' / 'case_details_0604.csv', index=False)

print("Scraping completed and data saved to case_details.csv")
# data preview 
df.head()

Scraping completed and data saved to case_details.csv


,year,url,id,syllabus,separate_opinion,endnotes,date,case_NO,lawyer_for_plaintiff,writer,decision,concur,plaintiffs,defendants,lawyer_for_defendant
0,1901,https://chanrobles.com/cralaw/1901januarydecis...,1,N/A,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1901,https://chanrobles.com/cralaw/1901januarydecis...,2,N/A,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1901,https://chanrobles.com/cralaw/1901januarydecis...,3,N/A,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1901,https://chanrobles.com/cralaw/1901januarydecis...,4,1. CONTEMPT OF COURT; TREATMENT OF WITNESS. — ...,False,False,"August 8, 1901",12,Claudio Gabriel,"SMITH, J.",It appears from the record in this matter that...,"s so ordered, with costs de oficio. Arellano, ...",NaN,NaN,NaN
4,1901,https://chanrobles.com/cralaw/1901januarydecis...,5,1. PARTNERSHIP; ACCOUNTING. — Any partner may ...,False,False,"August 24, 1901",26,Charles C . Cohn,"COOPER, J.",This is an appeal from the judgment of the Cou...,"s on appeal taxed to the Appellant . Arellano,...",WALTER JACKSON,"PAUL BLUM, ET AL.",Wm. J . Rohde


In [236]:
# drop rows if case_NO is missing
print(f"Before droping, the data has '{df.shape[0]}' rows")
df = df.dropna(subset=['case_NO'])
print(f"After droping, the data has '{df.shape[0]}' rows")


Before droping, the data has '22950' rows
After droping, the data has '9862' rows


In [237]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9862 entries, 3 to 22663
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   year                  9862 non-null   int64 
 1   url                   9862 non-null   object
 2   id                    9862 non-null   int64 
 3   syllabus              9862 non-null   object
 4   separate_opinion      9862 non-null   bool  
 5   endnotes              9862 non-null   bool  
 6   date                  9733 non-null   object
 7   case_NO               9862 non-null   object
 8   lawyer_for_plaintiff  9290 non-null   object
 9   writer                9778 non-null   object
 10  decision              9798 non-null   object
 11  concur                9798 non-null   object
 12  plaintiffs            9709 non-null   object
 13  defendants            9709 non-null   object
 14  lawyer_for_defendant  8750 non-null   object
dtypes: bool(2), int64(2), object(11)
memo

In [238]:
# store the url list where the decision is mission
missing_url = df[(df['decision'].isnull()) | (df['syllabus']=="N/A")]['url'].tolist()

print(f"Number of missing decision: '{len(missing_url)}'")

Number of missing decision: '214'


In [239]:

# Scrapping the url for decision is empty. 
def scrape_missing_data(url_list):
    data = []
    missing_url_list = []
    for url in url_list:
        print(f"Scraping URL: {url}")
        
        # Define headers to mimic a browser request
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }

        try:
            # Send a GET request to the webpage with headers and timeout
            response = requests.get(url, headers=headers, timeout=10)
            
            # If status code 404, break the loop
            if response.status_code == 404:
                print(f"No more data available for {url}")
                break

            # If any other status code, continue to the next ID
            if response.status_code != 200:
                print(f"Failed to retrieve the webpage for {url}. Status code: {response.status_code}")
                continue
            pattern = re.compile(r'/cralaw/(\d{4})januarydecisions\.php\?id=(\d+)')

            # Search for the pattern in the URL
            match = pattern.search(url)

            # Extract and print the year and id if the pattern is found
            if match:
                year = match.group(1)
                id_ = match.group(2)

            # Parse the webpage content
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find the relevant section containing the case details
            main_content = soup.find('div', class_='topcontent')
            if not main_content:
                continue

            # Initialize variables to store case details
            case_details = {'year': year, 'url': url, 'id': id_}

            # Parse the HTML content with BeautifulSoup

            # Find the <p> tag with align="right" and style containing "margin"
            decision_tag = main_content.find(string="D E C I S I O N")

            if not decision_tag:
                missing_url_list.append(url)
                print(f"Decision not found for URL: {url}")
                continue

            # Find the next sibling tags after the second "DECISION"
            next_siblings = decision_tag.find_all_next('p', class_="MsoNormal", style=lambda value: 'margin' in value if value else False)

            # Extract and print the text from these <p> tags
            extracted_texts = [tag.get_text(strip=True) for tag in next_siblings]

            # Adding the text together
            case_details['decision'] = ' '.join(extracted_texts)
            
            data.append(case_details)

        except requests.exceptions.Timeout:
            print(f"Request timed out for URL: {url}")
            continue
        except requests.exceptions.RequestException as e:
            print(f"Failed to retrieve the webpage for URL: {url}. Error: {e}")
            continue

    return data, missing_url_list


In [240]:

# Scrape the missing data
missing_data = pd.DataFrame(scrape_missing_data(missing_url)[0])

missing_url_list = scrape_missing_data(missing_url)[1]

Scraping URL: https://chanrobles.com/cralaw/1904januarydecisions.php?id=72
Decision not found for URL: https://chanrobles.com/cralaw/1904januarydecisions.php?id=72
Scraping URL: https://chanrobles.com/cralaw/1904januarydecisions.php?id=131
Decision not found for URL: https://chanrobles.com/cralaw/1904januarydecisions.php?id=131
Scraping URL: https://chanrobles.com/cralaw/1905januarydecisions.php?id=2
Scraping URL: https://chanrobles.com/cralaw/1905januarydecisions.php?id=3
Scraping URL: https://chanrobles.com/cralaw/1905januarydecisions.php?id=4
Scraping URL: https://chanrobles.com/cralaw/1905januarydecisions.php?id=5
Scraping URL: https://chanrobles.com/cralaw/1905januarydecisions.php?id=6
Scraping URL: https://chanrobles.com/cralaw/1905januarydecisions.php?id=7
Scraping URL: https://chanrobles.com/cralaw/1905januarydecisions.php?id=8
Scraping URL: https://chanrobles.com/cralaw/1905januarydecisions.php?id=9
Scraping URL: https://chanrobles.com/cralaw/1905januarydecisions.php?id=10
Scr

In [242]:
# Create a dictionary from missing_data for quick lookup
missing_data['unique_id'] = missing_data['year'].astype(str) + missing_data['id'].astype(str)
df['unique_id'] = df['year'].astype(str) + df['id'].astype(str)

unique_id = missing_data['unique_id']
missing_dict = dict(zip(missing_data['unique_id'], missing_data['decision']))

# Fill in the decision value from missing_data if the url is in missing_url
for key in missing_dict: 
    df.loc[df['unique_id'] == key, 'decision'] = missing_dict[key]


print(f"Before adding missing data, the data has '{df[df['syllabus'].isnull()].shape[0]}' rows")



Before adding missing data, the data has '0' rows


In [243]:
df[df['unique_id'] == '19052']

,year,url,id,syllabus,separate_opinion,endnotes,date,case_NO,lawyer_for_plaintiff,writer,decision,concur,plaintiffs,defendants,lawyer_for_defendant,unique_id
1996,1905,https://chanrobles.com/cralaw/1905januarydecis...,2,N/A,False,False,"November 2, 1905",1207.,NaN,"ARELLANO, C.J.","ARELLANO,C.J.: 1.Dalmacio Arquiza died on the ...","Torres, Mapa, Johnson and Carson, JJ., concur.","PIA BASA, ET AL., Plaintiffs-Appellants,","JOSE CLARO ARQUIZA, ET AL., Defendants-Appelle...",NaN,19052


In [244]:
df.to_csv(data_path / 'cleaned_case_details_0604.csv', index=False)

### Data Cleaning for Scrapping Data

In [245]:
# Read the data
data = pd.read_csv(data_path / 'cleaned_case_details_0604.csv')

# Reorder the column
data = data[['unique_id', 'year', 'date', 'case_NO', 'plaintiffs', 'defendants', 'lawyer_for_plaintiff', 'lawyer_for_defendant', 'writer', 'syllabus', 'decision', 'concur', 'separate_opinion', 'endnotes', 'url']]
# Clean the column "concur"

# Change the datatype
data['unique_id'] = data['unique_id'].astype(str)
data['year'] = data['year'].astype(str)
data['date'] = data['date'].astype(str)
data['case_NO'] = data['case_NO'].astype(str)
data['concur'] = data['concur'].astype(str)

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9862 entries, 0 to 9861
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   unique_id             9862 non-null   object
 1   year                  9862 non-null   object
 2   date                  9862 non-null   object
 3   case_NO               9862 non-null   object
 4   plaintiffs            9709 non-null   object
 5   defendants            9709 non-null   object
 6   lawyer_for_plaintiff  9289 non-null   object
 7   lawyer_for_defendant  8749 non-null   object
 8   writer                9778 non-null   object
 9   syllabus              9699 non-null   object
 10  decision              9724 non-null   object
 11  concur                9862 non-null   object
 12  separate_opinion      9862 non-null   bool  
 13  endnotes              9862 non-null   bool  
 14  url                   9862 non-null   object
dtypes: bool(2), object(13)
memory usage: 1

In [246]:
# Clean the column "concur"
for i, row in data.iterrows():
    text = row['concur']
    
    if len(text) == 100:
        
        text = text.replace("J. ,", "J.,")
        index = text.find("JJ.")

        # Find the position of the period before "JJ."
        period_index = text.rfind(". ", 0, index)

        # Extract the text between the period and "JJ."
        extracted_text = text[period_index + 2:index + 3]
        data.at[i, 'concur'] = extracted_text

data.head()

,unique_id,year,date,case_NO,plaintiffs,defendants,lawyer_for_plaintiff,lawyer_for_defendant,writer,syllabus,decision,concur,separate_opinion,endnotes,url
0,19014,1901,"August 8, 1901",12,NaN,NaN,Claudio Gabriel,NaN,"SMITH, J.",1. CONTEMPT OF COURT; TREATMENT OF WITNESS. — ...,It appears from the record in this matter that...,"Arellano, C.J., Cooper, Willard, Torres and La...",False,False,https://chanrobles.com/cralaw/1901januarydecis...
1,19015,1901,"August 24, 1901",26,WALTER JACKSON,"PAUL BLUM, ET AL.",Charles C . Cohn,Wm. J . Rohde,"COOPER, J.",1. PARTNERSHIP; ACCOUNTING. — Any partner may ...,This is an appeal from the judgment of the Cou...,"Arellano, C.J., Torres, Smith, Willard and Lad...",False,False,https://chanrobles.com/cralaw/1901januarydecis...
2,19016,1901,"August 26, 1901",17,DON LUCIANO CORDOBA,"WARNER, BARNES & CO.",Alberto Barrett,Josiah Mode Vale,"SMITH, J.",1. COMMON CARRIERS; BILLS OF LADING. — Where a...,This action was commenced in the Court of Firs...,"Arellano, C.J., Torres, Cooper, Willard, Ladd ...",False,False,https://chanrobles.com/cralaw/1901januarydecis...
3,19017,1901,"August 28, 1901",456,NaN,NaN,Eber C . Smith,"Colonel Grosbeck, United States Army","WILLARD, J.",1. HABEAS CORPUS; JURISDICTION. — Under Genera...,No judge of this Archipelago has at present ju...,"Arellano, C.J., Torres, Cooper, Mapa and Ladd,...",False,False,https://chanrobles.com/cralaw/1901januarydecis...
4,19018,1901,"September 9, 1901",100,AGUSTIN ASENCIO,FRANCISCO GUTIERREZ,Jose Robles Lahesa,Vicente Miranda,"WILLARD, J.",1. VENUE OF ACTIONS. — An action to collect re...,It has been plainly proven that at the time th...,"Arellano, C.J., Torres, Cooper, Mapa and Ladd,...",False,False,https://chanrobles.com/cralaw/1901januarydecis...


In [247]:
# save to csv
data.to_csv(data_path / 'final_case_details_0604_v1.csv', index=False)

## EDA

In [248]:

# Preliminary scrapping data results 
data = pd.read_csv(data_path / 'final_case_details_0604_v1.csv')

print(f"After droping, the data has '{data.shape[0]}' rows")

data.groupby('year').size()




After droping, the data has '9862' rows


year
1901     31
1902    160
1903    217
1904    140
1905    389
1906    499
1907    499
1908    422
1909    498
1910    226
1911    279
1912    234
1913     94
1914    238
1915    310
1916    313
1917    268
1918    214
1919    129
1920    107
1921    119
1922    168
1923    164
1924    200
1925    151
1926    210
1927    174
1928    196
1929    213
1930    201
1931    151
1932    151
1933    233
1934    257
1935    225
1936    141
1937    135
1938    164
1939    334
1940    278
1941    318
1942     94
1943     71
1944     33
1945     32
1946    182
dtype: int64

In [227]:
# how many columns that do not have year from 1901 to 1946
data['year'] = data['year'].astype(int)
data[(data['year'] < 1901) | (data['year'] > 1946)].shape[0]

0